# Load AutoLaparo

In [ ]:
import os
import sys
sys.path.append("../..")

import pandas as pd


from utils.io import load_yaml

preview_horizon = 1
seq_len = 15
frame_increment = 5
frames_between_clips = 1

server = "local"
server = load_yaml("../../config/servers.yml")[server]
database = "autolaparo_single_frames/AutoLaparo_Task2"
prefix = os.path.join(server["database"]["location"], database)
# pkl = f"23_03_03_pre_processed_frame_increment_{frame_increment}_frames_between_clips_1_log.pkl"
pkl = f"23_03_03_motion_label_window_1_frame_increment_{frame_increment}_frames_between_clips_1_log_test_train.pkl"

vid_df = pd.read_pickle(os.path.join(prefix, pkl))

# sort!
vid_df[["vid", "frame"]] = vid_df[["vid", "frame"]].astype(float)
vid_df = vid_df.sort_values(by=["vid", "frame"]).reset_index(drop=True)
vid_df[["vid", "frame"]] = vid_df[["vid", "frame"]].astype(int)

label_df = pd.read_csv(os.path.join(prefix, "laparoscope_motion_label.csv"))
label_df.Clip = label_df.Clip.apply(lambda x: x-1)

print(vid_df)
print(label_df)

# remove data that is irrelevant to autolaparo
length = len(vid_df[vid_df["vid"] == 0])
print(f"initial length: {length}")

# get last half plus sequence length
# print(len(vid_df)) 250 frames, of which we take half. Increment is missing due to pre-processing
# vid_df = vid_df.groupby("vid").tail(int(length/2) + seq_len - preview_horizon).reset_index(drop=True)
# vid_df = vid_df.groupby("vid").iloc[125 - seq_len*frame_increment:200].reset_index(drop=True)
vid_df = vid_df[vid_df.frame >= 120 - (seq_len-1)*frame_increment]
vid_df = vid_df[vid_df.frame < 120 + (seq_len-5)*frame_increment].reset_index(drop=True)

print(vid_df)


length = len(vid_df[vid_df["vid"] == 0])
print(f"length after: {length}")

# from readme
motion_dict = {
    0: "Static",
    1: "Up",
    2: "Down",
    3: "Left",
    4: "Right",
    5: "Zoom-in",
    6: "Zoom-out",
}

# Load Homography Predictor

In [ ]:
import torch
from lightning_modules.homography_imitation import ConvHomographyPredictorModule


def cholec80(resnet: int=18):
    if resnet == 18:
        checkpoint_prefix = "/media/martin/Samsung_T5/logs/miccai/final/cholec80/resnet18/version_2"
        checkpoint = "checkpoints/epoch=39-step=78400.ckpt"
    elif resnet == 34:
        checkpoint_prefix = "/media/martin/Samsung_T5/logs/miccai/final/cholec80/resnet34/version_2"
        checkpoint = "checkpoints/epoch=67-step=133280.ckpt"
    elif resnet == 50:
        checkpoint_prefix = "/media/martin/Samsung_T5/logs/miccai/final/cholec80/resnet50/version_2"
        checkpoint = "checkpoints/epoch=62-step=123480.ckpt"
    return checkpoint_prefix, checkpoint

def heichole(resnet: int=18):
    if resnet == 18:
        checkpoint_prefix = "/media/martin/Samsung_T5/logs/miccai/final/heichole/resnet18/version_1"
        checkpoint = "checkpoints/epoch=32-step=11715.ckpt"
    elif resnet == 34:
        checkpoint_prefix = "/media/martin/Samsung_T5/logs/miccai/final/heichole/resnet34/version_3"
        checkpoint = "checkpoints/epoch=48-step=17395.ckpt"
    elif resnet == 50:
        checkpoint_prefix = "/media/martin/Samsung_T5/logs/miccai/final/heichole/resnet50/version_1"
        checkpoint = "checkpoints/epoch=36-step=13135.ckpt"
    return checkpoint_prefix, checkpoint

def autolaparo(resnet: int=18):
    if resnet == 18:
        checkpoint_prefix = "/media/martin/Samsung_T5/logs/miccai/final/autolaparo/resnet18/version_0"
        checkpoint = "checkpoints/epoch=26-step=3159.ckpt"
    elif resnet == 34:
        checkpoint_prefix = "/media/martin/Samsung_T5/logs/miccai/final/autolaparo/resnet34/version_0"
        checkpoint = "checkpoints/epoch=41-step=4914.ckpt"
    elif resnet == 50:
        checkpoint_prefix = "/media/martin/Samsung_T5/logs/miccai/final/autolaparo/resnet50/version_0"
        checkpoint = "checkpoints/epoch=48-step=5733.ckpt"
    return checkpoint_prefix, checkpoint

def phantom(resnet: int=18):
    if resnet == 18:
        checkpoint_prefix = "/media/martin/Samsung_T5/logs/miccai/final/phantom/resnet18/version_0"
        checkpoint = "checkpoints/epoch=138-step=834.ckpt"
    elif resnet == 34:
        checkpoint_prefix = "/media/martin/Samsung_T5/logs/miccai/final/phantom/resnet34/version_0"
        checkpoint = "checkpoints/epoch=62-step=378.ckpt"
    elif resnet == 50:
        checkpoint_prefix = "/media/martin/Samsung_T5/logs/miccai/final/phantom/resnet50/version_0"
        checkpoint = "checkpoints/epoch=39-step=240.ckpt"
    return checkpoint_prefix, checkpoint

checkpoint_prefix, checkpoint = cholec80(34)
# checkpoint_prefix, checkpoint = heichole(50)
# checkpoint_prefix, checkpoint = phantom(18)

config = load_yaml(os.path.join(checkpoint_prefix, "config.yml"))

device = "cpu"
if torch.cuda.is_available():
    device = "cuda"

predictor = ConvHomographyPredictorModule.load_from_checkpoint(
    os.path.join(checkpoint_prefix, checkpoint), **config["model"]
)
predictor.to(device)
predictor = predictor.eval()
predictor.freeze()

# Load Homography Estimator and Taylor

In [ ]:
from utils.processing import TaylorHomographyPrediction
from lightning_modules.homography_regression import DeepImageHomographyEstimationModuleBackbone

def ae_cai():
    checkpoint_prefix = "/media/martin/Samsung_T5/logs/ae_cai/resnet/48/25/34/version_0"
    checkpoint = "checkpoints/epoch=99-step=47199.ckpt"
    return checkpoint_prefix, checkpoint


checkpoint_prefix_estimator, checkpoint_estimator = ae_cai()

estimator_config = load_yaml(os.path.join(checkpoint_prefix_estimator, "config.yml"))

estimator = DeepImageHomographyEstimationModuleBackbone.load_from_checkpoint(
    os.path.join(checkpoint_prefix_estimator, checkpoint_estimator), **estimator_config["model"]
)
estimator.to(device)
estimator = estimator.eval()
estimator.freeze()

taylor_predictor = TaylorHomographyPrediction(order=1)

# Predict Homographies

# Display Images

In [ ]:
from torch.utils.data import DataLoader
from datasets import ImageSequenceMotionLabelDataset, ImageSequenceDataset
import tqdm

from utils.transforms import dict_list_to_augment

import PIL
from IPython.display import display, clear_output
from kornia import tensor_to_image

output_path = "/media/martin/Samsung_T5/23_02_20_miccai_measurements/eval/23_03_06_trained_model_pred_on_autolaparo"

transforms = dict_list_to_augment([
    {"chance": 1.0, "module": "imgaug.augmenters", "type": "Resize", "kwargs": {"size": {"height": 240, "width": 320}}}
])

ds = ImageSequenceDataset(
    df=vid_df,
    prefix=prefix,
    seq_len=seq_len,
    frame_increment=frame_increment,
    frames_between_clips=seq_len*frame_increment,
    geometric_transforms=transforms,
)

dl = DataLoader(ds, batch_size=1, shuffle=False, num_workers=0)

results = []
for batch in tqdm.tqdm(dl):
    # pre-process
    imgs, imgs_tf, frame_idcs, vid_idcs = batch
    B, T, C, H, W = imgs.shape
    preview_horizon_imgs = imgs[:, :-preview_horizon]
    for img in preview_horizon_imgs[0]:
        display(PIL.Image.fromarray(tensor_to_image(img, keepdim=False)))
        clear_output(wait=True)


## Run Predction

In [ ]:
import os
from torch.utils.data import DataLoader
from datasets import ImageSequenceMotionLabelDataset, ImageSequenceDataset
import tqdm
import numpy as np

from utils.transforms import dict_list_to_augment
from utils.viz import create_blend_from_four_point_homography
from utils.processing import classify_duv_motion, frame_pairs

output_path = "/media/martin/Samsung_T5/23_02_20_miccai_measurements/eval/23_03_07_trained_model_pred_on_autolaparo"

transforms = dict_list_to_augment([
    {"chance": 1.0, "module": "imgaug.augmenters", "type": "Resize", "kwargs": {"size": {"height": 240, "width": 320}}}
])

delta = 0
frame_increment = 5

ds = ImageSequenceDataset(
    df=vid_df,
    prefix=prefix,
    seq_len=seq_len + delta,
    frame_increment=frame_increment,
    frames_between_clips=frame_increment, # frames_between_clips,
    geometric_transforms=transforms,
    random_frame_offset=False
)

dl = DataLoader(ds, batch_size=16, shuffle=False, num_workers=4, drop_last=True)

results = []
sigma = 11.
for batch in tqdm.tqdm(dl):
    # pre-process
    imgs, imgs_tf, frame_idcs, vid_idcs = batch
    B, T, C, H, W = imgs.shape
    imgs = imgs.to(device).float() / 255.
    recall_horizon_imgs = imgs[:, :-delta-preview_horizon]
    recall_horizon_imgs = recall_horizon_imgs.reshape(B, -1, H, W)
    recall_horizon_imgs = recall_horizon_imgs

    # inference
    with torch.no_grad():
        duvs_predicted = predictor(recall_horizon_imgs)

    # estimate camera motion for taylor
    imgs_i, imgs_ip1 = frame_pairs(recall_horizon_imgs.view(B, -1, C, H, W))
    with torch.no_grad():
        duvs_estimated = estimator(imgs_i.reshape(-1, C, H, W), imgs_ip1.reshape(-1, C, H, W))
        duvs_estimated = duvs_estimated.view(B, T-1-preview_horizon, 4, 2)

    # taylor predict camera motion
    duvs_taylor_predicted = taylor_predictor(duvs_estimated.cpu())[:, -preview_horizon:].squeeze()

    # visualize
    # recall_horizon_imgs = recall_horizon_imgs.reshape(B, -1, C, H, W)
    # blends = create_blend_from_four_point_homography(recall_horizon_imgs[:, -1], imgs[:, -1], duvs)

    # import PIL
    # from IPython.display import display, clear_output, TextDisplayObject
    # from kornia import tensor_to_image
    # from kornia.geometry import resize
    # import numpy as np
    # for blend, vid_idx in zip(blends, vid_idcs):
    #     motion = motion_dict[label_df[label_df["Clip"] == vid_idx.item()]["Label"].values[0]]
    #     display(PIL.Image.fromarray((tensor_to_image(resize(blend, [480, 640]), keepdim=False) * 255.).astype(np.uint8)))
    #     display(TextDisplayObject(motion).data)
    #     clear_output(wait=True)

    # break

    duvs_mpd = np.linalg.norm(duvs_predicted.cpu().numpy(), axis=-1).mean(axis=-1)

    # logs duvs with video label
    for duv_predicted, duv_taylor_predicted, duv_mpd, vid_idx in zip(duvs_predicted, duvs_taylor_predicted, duvs_mpd, vid_idcs):
        # or classify motion here! and keep only directed
        # if duv_mpd > 10: # discard noisy / undecisive motions for binary classification

        duv_predicted = duv_predicted.cpu().numpy().tolist()
        motion = classify_duv_motion(duv_predicted[0][0], duv_predicted[0][1], duv_predicted[1][0], duv_predicted[1][1], duv_predicted[2][0], duv_predicted[2][1], duv_predicted[3][0], duv_predicted[3][1], motion_threadhold=1.0*sigma)
        label = motion_dict[label_df[label_df["Clip"] == vid_idx.item()]["Label"].values[0]]

        result = {
            "vid": vid_idx.item(),
            "duv_predicted": np.nan,
            "duv_taylor_predicted": np.nan,
            "label": label,
        }

        if motion != "static":
            result["duv_predicted"] = duv_predicted

        duv_taylor_predicted = duv_taylor_predicted.cpu().numpy().tolist()
        motion = classify_duv_motion(duv_taylor_predicted[0][0], duv_taylor_predicted[0][1], duv_taylor_predicted[1][0], duv_taylor_predicted[1][1], duv_taylor_predicted[2][0], duv_taylor_predicted[2][1], duv_taylor_predicted[3][0], duv_taylor_predicted[3][1], motion_threadhold=1.0*sigma)
        if motion != "static":
            result["duv_taylor_predicted"] = duv_taylor_predicted

        results.append(result)

results_df = pd.DataFrame(results)
name = "_".join(config["experiment"].split("/")[-2:])
print(name)
results_df.to_pickle(os.path.join(output_path, f"23_03_07_autolaparo_{name}.pkl"))
results_df.to_csv(os.path.join(output_path, f"23_03_07_autolaparo_{name}.csv"))
results_df


# Load and Classify Homographies

# Utilities

In [ ]:
import os
import sys
sys.path.append("../..")

from typing import List
import pandas as pd

from utils.processing import classify_duv_motion

datasets = ["cholec80"]
# backbones = ["resnet18", "resnet34", "resnet50"]
backbones = ["resnet34"]

def to_path(datset, backbone, prefix="/media/martin/Samsung_T5/23_02_20_miccai_measurements/eval"):
    return os.path.join(prefix, f"23_03_07_trained_model_pred_on_autolaparo/23_03_07_autolaparo_{datset}_{backbone}.pkl")

# Continuous Classification

In [ ]:
# duv to homography
import numpy as np
import cv2


shape = [240, 320]

# center point
p = np.array([int((shape[0]-1)/2), int((shape[1]-1)/2)]).astype(np.float32)

img_edges = np.array([
    [0, 0],
    [0, shape[1]],
    [shape[0], shape[1]],
    [shape[0], 0],
])

def center_point_classifier(duv: np.array) -> list:
    if np.isnan(duv).any():
        return [np.nan, np.nan]
    wrp_edges = img_edges + duv
    h = cv2.getPerspectiveTransform(img_edges.astype(np.float32), wrp_edges.astype(np.float32))
    
    # transform center under homography
    p_prime = cv2.perspectiveTransform(p.reshape(-1, 1, 2), h)
    
    # get delta
    duv_center = p_prime - p
    return duv_center[0][0].tolist()

def edge_classifier(duv: np.array) -> list:
    return duv.mean(axis=0).tolist()


for dataset in datasets:
    for backbone in backbones:
        print(backbone)
        df = pd.read_pickle(to_path(dataset, backbone))
        duvs_predicted_center = []
        duvs_taylor_predicted_center = []

        for idx, row in df.iterrows():
            duvs_predicted_center.append(center_point_classifier(np.array(row.duv_predicted)))
            duvs_taylor_predicted_center.append(center_point_classifier(np.array(row.duv_taylor_predicted)))
            # duvs_center.append(edge_classifier(row.duv))

        # log delta p
        df["duv_predicted_center"] = duvs_predicted_center
        df["duv_taylor_predicted_center"] = duvs_taylor_predicted_center

        print(df)

        df.to_csv(to_path(dataset, backbone).split(".")[0] + "_duv_center.csv")
        df.to_pickle(to_path(dataset, backbone).split(".")[0] + "_duv_center.pkl")
    break


## Load and Plot

In [ ]:
# violin plot
import os
import sys
sys.path.append("../..")

import seaborn
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
# plt.rcParams['text.usetex'] = True

import pandas as pd


datasets = ["cholec80"]
# backbones = ["resnet18", "resnet34", "resnet50"]
backbones = ["resnet18", "resnet34", "resnet50"]

backbone_rename = {
    "resnet18": "ResNet-18",
    "resnet34": "ResNet-34",
    "resnet50": "Resnet-50",
}

def to_path(datset, backbone, prefix="/media/martin/Samsung_T5/23_02_20_miccai_measurements/eval"):
    return os.path.join(prefix, f"23_03_07_trained_model_pred_on_autolaparo/23_03_07_autolaparo_{datset}_{backbone}_duv_center.pkl")

# target_labels = ["Up", "Down", "Left", "Right", "Zoom-in", "Zoom-out"]
target_labels = ["Up", "Down", "Left", "Right"]

seaborn.set_style("whitegrid")
# seaborn.set(font_scale=1.5, rc={'text.usetex' : True})
for dataset in datasets:
    for backbone in backbones:
        df = pd.read_pickle(to_path(dataset, backbone))

        # try:
        #     grouped = df.groupby("label")
        #     df.drop(grouped.get_group("Zoom-in").index, inplace=True)
        # except Exception as e:
        #     print(e)
        # try:
        #     grouped = df.groupby("label")
        #     df.drop(grouped.get_group("Zoom-out").index, inplace=True)
        # except Exception as e:
        #     print(e)
        try:
            grouped = df.groupby("label")
            df.drop(grouped.get_group("Static").index, inplace=True)
        except Exception as e:
            print(e)

        seaborn.set(font_scale=1.2)
        seaborn.set_palette("husl", n_colors=4)

        df["duv_predicted_x"] = df.duv_predicted_center.apply(lambda x: x[1]/120. if not np.isnan(x).any() else np.nan)
        df["duv_predicted_y"] = df.duv_predicted_center.apply(lambda x: -x[0]/160 if not np.isnan(x).any() else np.nan)
        df["duv_taylor_predicted_x"] = df.duv_taylor_predicted_center.apply(lambda x: x[1]/120. if not np.isnan(x).any() else np.nan)
        df["duv_taylor_predicted_y"] = df.duv_taylor_predicted_center.apply(lambda x: -x[0]/160 if not np.isnan(x).any() else np.nan)



        df.rename(columns={"duv_predicted_x": backbone_rename[backbone], "duv_taylor_predicted_x": "O(1)-Taylor", "label": "AutoLaparo"}, inplace=True)
        merged_df = pd.melt(df, id_vars=["AutoLaparo"], value_vars=[backbone_rename[backbone], "O(1)-Taylor"], var_name="Method", value_name="Predicted x-axis camera motion [a.u.]")
        seaborn.boxplot(x="Predicted x-axis camera motion [a.u.]", y="Method", data=merged_df, hue="AutoLaparo", order=["O(1)-Taylor", backbone_rename[backbone]], hue_order=target_labels, width=0.4, fliersize=2)
        plt.axvline(x=0, color="black", linestyle="--")
        plt.savefig(f"/media/martin/Samsung_T5/23_02_20_miccai_measurements/eval/23_03_07_autolaparo_{dataset}_{backbone}_duv_center_x.pdf", bbox_inches='tight')
        plt.savefig(f"/media/martin/Samsung_T5/23_02_20_miccai_measurements/eval/23_03_07_autolaparo_{dataset}_{backbone}_duv_center_x.png", bbox_inches='tight')
        plt.clf()

        # repeat for y
        df.rename(columns={backbone_rename[backbone]: "duv_predicted_x", "O(1)-Taylor": "duv_taylor_predicted_x"}, inplace=True)
        df.rename(columns={"duv_predicted_y": backbone_rename[backbone], "duv_taylor_predicted_y": "O(1)-Taylor"}, inplace=True)
        merged_df = pd.melt(df, id_vars=["AutoLaparo"], value_vars=[backbone_rename[backbone], "O(1)-Taylor"], var_name="Method", value_name="Predicted y-axis camera motion [a.u.]")
        seaborn.boxplot(x="Method", y="Predicted y-axis camera motion [a.u.]", data=merged_df, hue="AutoLaparo", order=["O(1)-Taylor", backbone_rename[backbone]], hue_order=target_labels, width=0.4, fliersize=2)
        plt.axhline(y=0, color="black", linestyle="--")
        plt.savefig(f"/media/martin/Samsung_T5/23_02_20_miccai_measurements/eval/23_03_07_autolaparo_{dataset}_{backbone}_duv_center_y.pdf", bbox_inches='tight')
        plt.savefig(f"/media/martin/Samsung_T5/23_02_20_miccai_measurements/eval/23_03_07_autolaparo_{dataset}_{backbone}_duv_center_y.png", bbox_inches='tight')
        plt.clf()


        # # seaborn.violinplot(x="duv_x", y="label", gridsize=200, data=df)
        # # seaborn.boxplot(x="duv_x", y="label", data=df[df.label.isin(["Left", "Right"])], width=0.2, palette={"Left": "C2", "Right": "C3"}, order=["Left", "Right"])
        # seaborn.boxplot(x="duv_predicted_x", y="label", data=df[["duv_predicted_x", "duv_taylor_predicted_x", "label"]][df.label.isin(["Left", "Right", "Up", "Down"])], width=0.2, order=["Left", "Right", "Up", "Down"])
        # # seaborn.boxplot(x="duv_taylor_predicted_x", y="label", data=df[["duv_taylor_predicted_x", "label"]].dropna()[df.label.isin(["Left", "Right", "Up", "Down"])], width=0.2, order=["Left", "Right", "Up", "Down"])
        # # plt.title("")
        # plt.xlabel("Predicted camera motion along x-axis / a.u.")
        # plt.ylabel("AutoLaparo label")
        # plt.axvline(0, linestyle="--", color="black")
        # plt.xlim([-0.9, 0.9])
        # # plt.show()
        # plt.savefig(f"/media/martin/Samsung_T5/23_02_20_miccai_measurements/eval/23_03_07_autolaparo_{dataset}_{backbone}_duv_center_x.pdf", bbox_inches='tight')
        # plt.savefig(f"/media/martin/Samsung_T5/23_02_20_miccai_measurements/eval/23_03_07_autolaparo_{dataset}_{backbone}_duv_center_x.png", bbox_inches='tight')
        # plt.clf()
        # # seaborn.violinplot(x="label", y="duv_y", gridsize=200, data=df)
        # # seaborn.boxplot(x="label", y="duv_y", data=df[df.label.isin(["Up", "Down"])], width=0.2, palette={"Up": "C0", "Down": "C1"}, order=["Up", "Down"])
        # seaborn.boxplot(x="label", y="duv_predicted_y", data=df[["duv_predicted_y", "label"]][df.label.isin(["Left", "Right", "Up", "Down"])], width=0.2, order=["Left", "Right", "Up", "Down"])
        # plt.xlabel("AutoLaparo label") 
        # plt.ylabel("Predicted camera motion along y-axis / a.u.")
        # plt.axhline(0, linestyle="--", color="black")
        # plt.ylim([-0.5, 0.5])
        # # plt.show()
        # plt.savefig(f"/media/martin/Samsung_T5/23_02_20_miccai_measurements/eval/23_03_07_autolaparo_{dataset}_{backbone}_duv_center_y.pdf", bbox_inches='tight')
        # plt.savefig(f"/media/martin/Samsung_T5/23_02_20_miccai_measurements/eval/23_03_07_autolaparo_{dataset}_{backbone}_duv_center_y.png", bbox_inches='tight')
        # plt.clf()

        # seaborn.boxplot(x="duv_taylor_predicted_x", y="label", data=df[["duv_taylor_predicted_x", "label"]].dropna()[df.label.isin(["Left", "Right", "Up", "Down"])], width=0.2, order=["Left", "Right", "Up", "Down"])
        # # plt.title("")
        # plt.xlabel("Predicted camera motion along x-axis / a.u.")
        # plt.ylabel("AutoLaparo label")
        # plt.axvline(0, linestyle="--", color="black")
        # plt.xlim([-0.9, 0.9])
        # # plt.show()
        # plt.savefig(f"/media/martin/Samsung_T5/23_02_20_miccai_measurements/eval/23_03_07_autolaparo_taylor_{dataset}_{backbone}_duv_center_x.pdf", bbox_inches='tight')
        # plt.savefig(f"/media/martin/Samsung_T5/23_02_20_miccai_measurements/eval/23_03_07_autolaparo_taylor_{dataset}_{backbone}_duv_center_x.png", bbox_inches='tight')
        # plt.clf()
        # # seaborn.violinplot(x="label", y="duv_y", gridsize=200, data=df)
        # # seaborn.boxplot(x="label", y="duv_y", data=df[df.label.isin(["Up", "Down"])], width=0.2, palette={"Up": "C0", "Down": "C1"}, order=["Up", "Down"])
        # seaborn.boxplot(x="label", y="duv_taylor_predicted_y", data=df[["duv_taylor_predicted_y", "label"]].dropna()[df.label.isin(["Left", "Right", "Up", "Down"])], width=0.2, order=["Left", "Right", "Up", "Down"])
        # plt.xlabel("AutoLaparo label") 
        # plt.ylabel("Predicted camera motion along y-axis / a.u.")
        # plt.axhline(0, linestyle="--", color="black")
        # plt.ylim([-0.5, 0.5])
        # # plt.show()
        # plt.savefig(f"/media/martin/Samsung_T5/23_02_20_miccai_measurements/eval/23_03_07_autolaparo_taylor_{dataset}_{backbone}_duv_center_y.pdf", bbox_inches='tight')
        # plt.savefig(f"/media/martin/Samsung_T5/23_02_20_miccai_measurements/eval/23_03_07_autolaparo_taylor_{dataset}_{backbone}_duv_center_y.png", bbox_inches='tight')
        # plt.clf()



        # break
    # break


In [ ]:
# scatter plot
import os
import sys
sys.path.append("../..")

import pandas as pd

datasets = ["heichole"]
# backbones = ["resnet18", "resnet34", "resnet50"]
backbones = ["resnet34"]

def to_path(datset, backbone, prefix="/media/martin/Samsung_T5/23_02_20_miccai_measurements/eval"):
    return os.path.join(prefix, f"23_03_07_trained_model_pred_on_autolaparo/23_03_07_autolaparo_{datset}_{backbone}_duv_center.pkl")

target_labels = ["Up", "Down", "Left", "Right", "Static"]

for dataset in datasets:
    for backbone in backbones:
        df = pd.read_pickle(to_path(dataset, backbone))
        for label in target_labels:
            sub_df = df[df["label"] == label]
            
            # plot duv scatter
            import matplotlib.pyplot as plt
            plt.scatter(sub_df.duv_center.apply(lambda x: x[1]), sub_df.duv_center.apply(lambda x: -x[0])) # opencv coordinates to standard right handed coordinates
            plt.title(f"{dataset} {backbone} {label}")
            plt.xlim(-20, 20)
            plt.ylim(-20, 20)
            plt.show()

            print("x_mean: ", sub_df.duv_center.apply(lambda x: x[1]).mean(), " x_std: ", sub_df.duv_center.apply(lambda x: x[1]).std())
            print("y_mean: ", sub_df.duv_center.apply(lambda x: -x[0]).mean(), " y_std: ", sub_df.duv_center.apply(lambda x: -x[0]).std())
        break
    break

# Discrete Classification

In [ ]:
def duv_label() -> List[str]:
    return [f"duv_{i}_{j}" for i in range(4) for j in range(2)]

def split_duv(df: pd.DataFrame) -> pd.DataFrame:
    df_split = df
    df_split["duv"] = df["duv"].apply(
        lambda x: np.array(x).flatten()
    )
    df_split = pd.DataFrame(
        df_split["duv"].to_list(), columns=duv_label()
    )

    df_split = pd.concat(
        [df.reset_index(drop=True), df_split], axis=1
    )

    return df_split

# from readme
motion_dict_other = {
    0: "static",
    1: "up",
    2: "down",
    3: "left",
    4: "right",
    5: "zoom_in",
    6: "zoom_out",
}

true_cnt = 0
false_cnt = 0

for dataset in datasets:
    for backbone in backbones:
        df = pd.read_pickle(to_path(dataset, backbone))
        
        df = split_duv(df)
        df["label"] = df.apply(lambda x: classify_duv_motion(x.duv_0_0, x.duv_0_1, x.duv_1_0, x.duv_1_1, x.duv_2_0, x.duv_2_1, x.duv_3_0, x.duv_3_1, motion_threadhold=7), axis=1)

        for name, group in df.groupby(by="vid"):
            autolaparo_label = motion_dict_other[label_df[label_df["Clip"] == name]["Label"].values[0]]
            labels = group.label.value_counts()
            try:
                labels = labels.drop(index=["static"])
            except:
                pass
            try:
                labels = labels.drop(index=["mixture"])
            except:
                pass
            if len(labels) > 1:
                # print(labels.index[0], autolaparo_label)
                if labels.index[0] == autolaparo_label or labels.index[1] == autolaparo_label: # first most common
                    true_cnt += 1
                else:
                    false_cnt += 1

        print(backbone, true_cnt, false_cnt, true_cnt / (true_cnt + false_cnt))
    break